# 本节内容摘要



### 2.基础优化算法

- **梯度下降**

  - 当模型没有显示解的时候，应用梯度下降法逼近最优解。
  - 梯度下降法的具体步骤：
    - 挑选一个初始值$\omega_0$
    - 重复迭代参数，迭代公式为：$\omega_t=\omega_{t-1}-\lambda\frac{\partial l}{\partial\omega_{t-1} } $
      - **$-\frac{\partial l}{\partial\omega_{t-1}}$为函数值下降最快的方向，学习率$\lambda$为学习步长。**
  - 选择学习率
    - 学习率$\lambda$为学习步长，代表了沿负梯度方向走了多远，这是超参数（人为指定的的值，不是训练得到的）
    - 学习率不能太大，也不能太小，需要选取适当。

- **小批量随机梯度下降**

  - 在整个训练集上算梯度太贵了

    - 在实际应用中，很少直接应用梯度下降法，这是因为每次更新都需要计算训练集上所有的样本，耗费时间太长。
    - 一个深度神经网络模型，迭代一次可能需要数分钟甚至数小时。

  - 为了减少运算代价，我们可以随机采样b个样本$i_1,i_2,...,i_b$来近似损失，损失函数为：

    ​	$\frac{1}{b}\sum_{i\in I_b}l(x_i,y_i,\omega)$ , 

    其中**b是批量大小(batch size)，也是超参数**

  - **选择批量大小**
    
    - b也不能太大：内存消耗增加；浪费计算资源，一个极端的情况是可能会重复选取很多差不多的样本，浪费计算资源
    - b也不能太小：每次计算量太小，很难以并行，不能最大限度利用GPU资源

- **总结**

  - 梯度下降通过不断**沿着梯度反方向**更新参数求解
  - 小批量随机梯度下降是深度学习默认的求解算法（简单，稳定）
  - **两个重要的超参数：批量大小（batch size），学习率（lr）**

# 讲义

In [1]:
from IPython.display import display, IFrame

# 使用IFrame将PDF文件嵌入到Notebook中
pdf_path = "../PPT/part-0_9.pdf"
display(IFrame(pdf_path, width=1100, height=600))


# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [3]:
# 调用框架现有的API来读取数据
def load_array(data_arrays,batch_size,is_train=True):
    """构造一个Pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays) # dataset相当于Pytorch的Dataset。一个星号*，表示对list解开入参。      
    return data.DataLoader(dataset,batch_size,shuffle=is_train) # 返回的是从dataset中随机挑选出batch_size个样本出来     

batch_size = 10
data_iter = load_array((features,labels),batch_size) # 返回的数据的迭代器
next(iter(data_iter))

[tensor([[ 0.0062, -0.8600],
         [-0.5421,  0.5865],
         [-0.2224,  0.3530],
         [ 0.1049,  1.1843],
         [-0.5222, -0.9637],
         [ 0.7598,  1.0531],
         [-0.1623, -0.3923],
         [ 0.5898,  1.2558],
         [-1.2157,  0.0183],
         [-0.4795, -0.0923]]),
 tensor([[7.1272],
         [1.1257],
         [2.5582],
         [0.3801],
         [6.4359],
         [2.1436],
         [5.2343],
         [1.1127],
         [1.7208],
         [3.5446]])]

使用框架的预定义好的层

In [4]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [6]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter: # 从DataLoader里面一次一次把所有数据拿出来
        l = loss(net(X),y) # net(X) 为计算出来的线性回归的预测值
        trainer.zero_grad() # 梯度清零
        l.backward()
        trainer.step()  # SGD优化器优化模型
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000218
epoch 2, loss 0.000109
epoch 3, loss 0.000110


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0019, -0.0006])
b的估计误差： tensor([-9.9182e-05])
